# 特征工程

本文件主要实现以下内容：
* **特征转换**
    * 文本特征：使用`TF-IDF`转换文本特征
    * 数值特征：使用`StandScaler`进行归一化
    * 有序分类型变量：使用`OrdinalEncoder`进行编码
    * 分类型变量：使用`OneHotEncoder`进行编码
* **创建预处理PipeLine**

In [1]:
import joblib
import pathlib
import numpy as np
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

processed_data_dir = pathlib.Path("../dataset/processed")
model_dir = pathlib.Path("../app/models")

In [2]:
df = pd.read_feather(processed_data_dir / "processed_data.feather")
df.sample(n=2, random_state=42)

,location,employment_type,industry,fraudulent,department,telecommuting,has_company_logo,has_questions,required_experience,required_education,function,text,salary,text_length
191,sh,Missing,Missing,1,Missing,-1.0,-1.0,-1.0,Missing,Missing,Missing,人事 经理 主管 4s店 直招 人事行政 岗位职责 公司 人工成本 行政 费用 预算 管理 ...,8500.0,480
312,Lake Kelly,Internship,Finance,1,Missing,-1.0,-1.0,-1.0,Missing,Missing,Missing,investment banker corporate value recently ear...,70865.5,198


## 特征转换(Feature Transformer)

In [3]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer

### Ordinal Encoding

`Ordinal Encoding`（顺序编码） 是一种将类别型数据转换为数值型数据的编码方式。它适用于类别之间具有内在顺序关系的情况，即类别的顺序是有意义的

In [4]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder(
    handle_unknown="use_encoded_value",
    unknown_value=-1
)
ordinal_columns = ["required_education", "required_experience"]
ordinal_encoder

OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

### OneHot Encoding

In [5]:
from sklearn.preprocessing import OneHotEncoder

onthot_columns = ["employment_type", "function", "location", "department", "industry"]
onthot_encoder = OneHotEncoder(handle_unknown="ignore")
onthot_encoder

OneHotEncoder(handle_unknown='ignore')

### MixMaxScaler

In [6]:
from sklearn.preprocessing import MinMaxScaler

num_columns = ["salary", "text_length"]
scaler = MinMaxScaler(feature_range=(0, 1))
scaler

MinMaxScaler()

### TF-IDF向量化

**TF-IDF（Term Frequency-Inverse Document Frequency）** 是一种常用于文本挖掘的特征提取方法，目的是评估一个词对于某一文本或一组文本（语料库）的重要程度
* **TF（Term Frequency）**: 词频，指的是某个词在文档中出现的次数
$$
TF(t)=\frac{\text{某个词t在文档中出现的次数}}{\text{文档中的总词数}}
$$
* **IDF（Inverse Document Frequency）**: 逆文档频率，指的是一个词在语料库中出现的稀有程度
$$
IDF(t)=\log{\frac{总文档数}{包含词t的文档+1}}
$$
* TF-IDF 计算: 最终的 $TF-IDF$ 权重是 $TF$ 和 $IDF$ 的乘积，表示一个词在某一文档中的重要性

In [7]:
def load_stopwords(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            stopwords = {line.strip().lower() for line in f if line.strip()}
        return stopwords
    except FileNotFoundError:
        print(f"错误：文件 {file_path} 未找到！")
        return set()

In [8]:
import spacy

# 加载Spacy模型与停用词
nlp_zh = spacy.load("zh_core_web_sm")
nlp_en = spacy.load("en_core_web_sm")
en_stopwords = nlp_en.Defaults.stop_words
zh_stopwords = nlp_zh.Defaults.stop_words
extra_stopwords = load_stopwords("../dataset/external/stopwords")
stopwords = en_stopwords | zh_stopwords | extra_stopwords

In [9]:
tfidf = TfidfVectorizer(
    ngram_range=(1, 2), 
    token_pattern=r"(?u)\b[a-zA-Z]{2,}\b",
    stop_words=list(stopwords),
    max_features=800,
    max_df=0.4, # 忽略出现在超过 30% 文档中的词
    analyzer="word",
    min_df=0.1, # 忽略出现在低于 10% 文档中的词
    sublinear_tf=True,
    encoding="utf-8",
)
tfidf

TfidfVectorizer(max_df=0.4, max_features=800, min_df=0.1, ngram_range=(1, 2),
                stop_words=['ｃ］', '㈧', '不可抗拒', 'leg', '豁然', 'himself', '个别',
                            '着呢', 'plant', '怎么', '不单', '但愿', '从优', 'pretty',
                            '［①⑧］', 'election', '通过', '如次', 'road', '尔', '勃然',
                            '上去', '促进', '较比', '/', '长线', 'walshmouth', 'sit',
                            'discussion', 'office', ...],
                sublinear_tf=True, token_pattern='(?u)\\b[a-zA-Z]{2,}\\b')

### 创建预处理转换器

```python
class sklearn.compose.ColumnTransformer(
    transformers, *, remainder='drop', 
    sparse_threshold=0.3, n_jobs=None, 
    transformer_weights=None, verbose=False
)
```

* `transformers`：类型为`Sequence[tuple]`，其功能为定义不同特征列对应的转换器
    * `name`：转换器的名称
    * `transformer`：转换器对象
    * `columns`：选择列的标识
* `remainder`：控制未被 `transformers` 指定的列的处理方式
    * `drop`：直接丢弃未被选择的列
    * `passthrough`：保留未被选择的列，不做任何转换
* `sparse_threshold`：控制输出矩阵的稀疏性，若所有转换器输出的稀疏矩阵的总密度低于此阈值，则最终输出为稀疏矩阵；否则转换为密集矩阵
* `verbose_feature_names_out`：控制输出特征名称的生成规则
    * `True`：在特征名前添加转换器名称
    * `False`：直接使用原始特征名
* `n_jobs`：设置并行运行的作业数
    * `None`：单线程运行
    * `-1`：全部使用 CPU 运行
    * 整数：指定使用的 CPU 数量

In [10]:
preprocessor = ColumnTransformer(
    transformers=[
        ("TfidfVectorizer", tfidf, "text"),
        ("OrdinalEncoder", ordinal_encoder, ordinal_columns), 
        ("OneHotEncoder", onthot_encoder, onthot_columns),
        ("MinMaxScaler", scaler, num_columns),  
    ],
        remainder="drop",
        verbose=False,
        n_jobs=-1
)
preprocessor

ColumnTransformer(n_jobs=-1,
                  transformers=[('TfidfVectorizer',
                                 TfidfVectorizer(max_df=0.4, max_features=800,
                                                 min_df=0.1, ngram_range=(1, 2),
                                                 stop_words=['ｃ］', '㈧', '不可抗拒',
                                                             'leg', '豁然',
                                                             'himself', '个别',
                                                             '着呢', 'plant',
                                                             '怎么', '不单', '但愿',
                                                             '从优', 'pretty',
                                                             '［①⑧］', 'election',
                                                             '通过', '如次', 'road',
                                                             '尔', '勃然', '上去',
                                                             '促进', '较比', '/',
                                                             '长线', 'walshmouth',
                                                             'sit',
                                                             'discussion',
                                                             'office', ...],
                                                 sublin...
                                                 token_pattern='(?u)\\b[a-zA-Z]{2,}\\b'),
                                 'text'),
                                ('OrdinalEncoder',
                                 OrdinalEncoder(handle_unknown='use_encoded_value',
                                                unknown_value=-1),
                                 ['required_education', 'required_experience']),
                                ('OneHotEncoder',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['employment_type', 'function', 'location',
                                  'department', 'industry']),
                                ('MinMaxScaler', MinMaxScaler(),
                                 ['salary', 'text_length'])])

## 保存数据

### 保存`preprocessor`

In [11]:
joblib.dump(
    preprocessor, 
    processed_data_dir / "preprocessor.joblib", 
    compress=5
)

['../dataset/processed/preprocessor.joblib']

### 保存特征与属性

In [12]:
inputs = df.drop(columns=["fraudulent"])
target = df["fraudulent"]
joblib.dump(
    inputs, 
    processed_data_dir / "inputs.joblib", 
    compress=5
)
joblib.dump(
    target,
    processed_data_dir / "target.joblib",
    compress=5
)

['../dataset/processed/target.joblib']